# Add Crime Stats to Database

In [1]:
DB_NAME = 'property_data.db'

In [ ]:
import pandas as pd

path = 'local_crime_data/CIUS_Table_8_Offenses_Known_to_Law_Enforcement_by_State_by_City_2024.xlsx'

# Read with correct header row and drop footnotes/non-city rows
df = pd.read_excel(path, header=3)
df = df[df['City'].notna()].copy()

# Normalize column names
df.columns = (
    df.columns.str.replace('\n', ' ', regex=False)
              .str.replace('-', ' ', regex=False)
              .str.replace(r'\s+', ' ', regex=True)
              .str.strip()
              .str.lower()
)

# Coerce needed numerics
for col in ['population', 'violent crime', 'property crime']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Per-100k rates
pop = df['population'].replace({0: pd.NA})
df['violent_crime_rate_per_100k'] = (df['violent crime'] / pop) * 100_000
df['property_crime_rate_per_100k'] = (df['property crime'] / pop) * 100_000

# Output only requested columns
df_crime = (
    df[['state', 'city', 'violent_crime_rate_per_100k', 'property_crime_rate_per_100k']]
    .rename(columns={
        'violent_crime_rate_per_100k': 'violent_100k',
        'property_crime_rate_per_100k': 'property_100k',
    })
)
df_crime

,state,city,violent_100k,property_100k
0,ALABAMA,Abbeville,461.022632,1466.890193
1,ALABAMA,Adamsville,559.47458,3162.247628
2,ALABAMA,Addison,448.430493,1943.198804
3,ALABAMA,Alabaster,98.983959,1510.961018
4,ALABAMA,Albertville,236.640564,1759.745289
...,...,...,...,...
8982,WYOMING,Sheridan,136.301681,989.44924
8983,WYOMING,Shoshoni,0.0,0.0
8984,WYOMING,Thermopolis,36.376864,691.160422
8985,WYOMING,Torrington,496.794872,1057.692308


In [3]:
#1100 Cantrall Rd, Medford, OR, 97530

import duckdb

con = duckdb.connect(DB_NAME)




# select the full_address column from the properties table, print first row only
df_address = con.sql('select zpid, full_address, county from property_features').df()

con.close()
df_address

,zpid,full_address,county
0,252154044,"12312 SE Jack Rabbit Trl, Prineville, OR 97754",Crook County
1,84562162,"1291 NW Sjoren Ln, Hermiston, OR 97838",Umatilla County
2,220038839,"13369 Keasey Rd, Vernonia, OR 97064",Columbia County
3,79777487,"1463 Ferrier Road, Winlock, WA 98596",Lewis County
4,82370422,"14820 McAuley Rd, Yakima, WA 98908",Yakima County
...,...,...,...
773,84575618,"6165 W Langell Valley Rd, Bonanza, OR 97623",Klamath County
774,82334477,"7388 Upper Applegate Rd, Jacksonville, OR 97530",Jackson County
775,109156970,"851 SE Rancho Ln, Madras, OR 97741",Jefferson County
776,82339324,"8797 W Evans Creek Rd, Rogue River, OR 97537",Jackson County


In [4]:
import re
import pandas as pd

def normalize_state(s):
    if not isinstance(s, str):
        return None
    token = re.sub(r'[^A-Za-z]', '', s).upper()
    if token in {'OR', 'ORE', 'OREGON'}:
        return 'OREGON'
    if token in {'WA', 'WASH', 'WASHINGTON'}:
        return 'WASHINGTON'
    return None

def normalize_city(c):
    if not isinstance(c, str):
        return None
    c = c.replace('_', ' ').strip()
    return ' '.join(w.capitalize() for w in re.split(r'\s+', c) if w)

def normalize_county(c):
    if not isinstance(c, str):
        return None
    c = c.strip()
    # Remove "County" suffix if present (case insensitive)
    c = re.sub(r'\s+County\s*$', '', c, flags=re.IGNORECASE)
    return c.strip()

def extract_city_state(addr):
    if not isinstance(addr, str):
        return pd.Series({'city': None, 'state': None})
    
    parts = [p.strip() for p in addr.split(',')]
    if len(parts) < 3:
        return pd.Series({'city': None, 'state': None})
    
    # Find the state by looking for OR/WA pattern in any part
    state = None
    state_idx = None
    for i, part in enumerate(parts):
        normalized = normalize_state(part.split()[0])  # Take first word before zip/MLS
        if normalized:
            state = normalized
            state_idx = i
            break
    
    if state is None or state_idx == 0:
        return pd.Series({'city': None, 'state': None})
    
    # City should be the part right before the state
    city = normalize_city(parts[state_idx - 1])
    
    return pd.Series({'city': city, 'state': state})

# Test with the problematic address
test_addr = "939 Buckhorn Rd, Roseburg, OR 97470 MLS #16017"
result = extract_city_state(test_addr)
print(f"Test address: {test_addr}")
print(f"Extracted: {result}")

# Apply to your dataframe
df_address[['city', 'state']] = df_address['full_address'].apply(extract_city_state)
# Normalize county names
df_address['county'] = df_address['county'].apply(normalize_county)
df_address.head(2)

Test address: 939 Buckhorn Rd, Roseburg, OR 97470 MLS #16017
Extracted: city     Roseburg
state      OREGON
dtype: object


,zpid,full_address,county,city,state
0,252154044,"12312 SE Jack Rabbit Trl, Prineville, OR 97754",Crook,Prineville,OREGON
1,84562162,"1291 NW Sjoren Ln, Hermiston, OR 97838",Umatilla,Hermiston,OREGON


In [5]:
sql = """
select a.zpid, c.violent_100k, c.property_100k
, a.city, a.county, a.state
from df_address a
left join df_crime c on a.city = c.city and a.state = c.state

"""
final_crime = duckdb.sql(sql).df()
final_crime.head(2)

,zpid,violent_100k,property_100k,city,county,state
0,63345911,195.876108,1831.091834,Albany,Linn,OREGON
1,82335574,252.995370,1766.194090,Ashland,Jackson,OREGON


Lots of missing city data! Now use County crime stats to fill in missing data.
CIUS_Table_10_Offenses_Known_to_Law_Enforcement_by_State_by_Metropolitan_and_Nonmetropolitan_Counties_2024.xlsx

In [ ]:
import pandas as pd

path = 'local_crime_data/CIUS_Table_10_Offenses_Known_to_Law_Enforcement_by_State_by_Metropolitan_and_Nonmetropolitan_Counties_2024.xlsx'

# Read with correct header row and drop footnotes/non-city rows
county_crime = pd.read_excel(path, header=4)
county_crime = county_crime[county_crime['County'].notna()].copy()

# Normalize column names
county_crime.columns = (
    county_crime.columns.str.replace('\n', ' ', regex=False)
              .str.replace('-', ' ', regex=False)
              .str.replace(r'\s+', ' ', regex=True)
              .str.strip()
              .str.lower()
)

county_crime

,state,metropolitan/nonmetropolitan,county,violent crime,murder and nonnegligent manslaughter,rape,robbery,aggravated assault,property crime,burglary,larceny theft,motor vehicle theft,arson1
0,ALABAMA,Metropolitan County,Autauga,54.0,1.0,7.0,0.0,46.0,221.0,46.0,143.0,32.0,1.0
1,ALABAMA,Metropolitan County,Baldwin,137.0,0.0,4.0,0.0,133.0,170.0,23.0,142.0,5.0,2.0
2,ALABAMA,Metropolitan County,Bibb,34.0,0.0,5.0,0.0,29.0,78.0,26.0,43.0,9.0,0.0
3,ALABAMA,Metropolitan County,Blount,82.0,1.0,18.0,2.0,61.0,298.0,66.0,180.0,52.0,2.0
4,ALABAMA,Metropolitan County,Calhoun,238.0,2.0,5.0,0.0,231.0,192.0,48.0,144.0,0.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2439,WYOMING,Nonmetropolitan County,Sweetwater,11.0,0.0,4.0,1.0,6.0,87.0,8.0,61.0,18.0,2.0
2440,WYOMING,Nonmetropolitan County,Teton,5.0,0.0,0.0,0.0,5.0,21.0,5.0,16.0,0.0,0.0
2441,WYOMING,Nonmetropolitan County,Uinta,4.0,1.0,2.0,0.0,1.0,22.0,3.0,18.0,1.0,0.0
2442,WYOMING,Nonmetropolitan County,Washakie,2.0,0.0,0.0,0.0,2.0,7.0,0.0,6.0,1.0,0.0


In [ ]:
path_wash = 'local_crime_data/co-est2024-pop-53.xlsx'
path_oregon = 'local_crime_data/co-est2024-pop-41.xlsx'

population_wash = pd.read_excel(path_wash, header=3)
population_wash = population_wash.rename(columns={
    population_wash.columns[0]: 'county_state',
    population_wash.columns[6]: 'population'
}).dropna().iloc[1:]

population_oregon = pd.read_excel(path_oregon, header=3)
population_oregon = population_oregon.rename(columns={
    population_oregon.columns[0]: 'county_state',
    population_oregon.columns[6]: 'population'
}).dropna().iloc[1:]

# append the two dataframes
population = pd.concat([population_wash, population_oregon])[['county_state', 'population']]

# Extract county and state into separate columns
population[['county', 'state']] = population['county_state'].str.extract(r'\.(.+) County, (.+)')
# drop county_state column
population = population.drop(columns=['county_state'])
population['state'] = population['state'].str.upper()

population

,population,county,state
1,21039.0,Adams,WASHINGTON
2,22523.0,Asotin,WASHINGTON
3,218190.0,Benton,WASHINGTON
4,81228.0,Chelan,WASHINGTON
5,77958.0,Clallam,WASHINGTON
...,...,...,...
32,7522.0,Wallowa,OREGON
33,26507.0,Wasco,OREGON
34,611272.0,Washington,OREGON
35,1456.0,Wheeler,OREGON


In [8]:
# Join population with county_crime
county_crime_pop = county_crime.merge(population, on=['county', 'state'])
# calculate per 100k rates
county_crime_pop['violent_100k'] = (county_crime_pop['violent crime'] / county_crime_pop['population']) * 100_000
county_crime_pop['property_100k'] = (county_crime_pop['property crime'] / county_crime_pop['population']) * 100_000
# filter to state, county, population, violent_100k, property_100k
county_crime_pop = county_crime_pop[['state', 'county', 'population', 'violent_100k', 'property_100k']]

# join grok_mapping city with county_crime_pop
county_crime_pop = county_crime_pop.merge(df_address[['city', 'county', 'state']], on=['county', 'state'])
county_crime_pop

,state,county,population,violent_100k,property_100k,city
0,OREGON,Benton,98899.0,25.278314,138.525162,Monroe
1,OREGON,Benton,98899.0,25.278314,138.525162,Corvallis
2,OREGON,Benton,98899.0,25.278314,138.525162,Philomath
3,OREGON,Benton,98899.0,25.278314,138.525162,Corvallis
4,OREGON,Benton,98899.0,25.278314,138.525162,Alsea
...,...,...,...,...,...,...
668,WASHINGTON,Pacific,24245.0,28.871932,206.228088,Raymond
669,WASHINGTON,Pacific,24245.0,28.871932,206.228088,Naselle
670,WASHINGTON,Pacific,24245.0,28.871932,206.228088,Naselle
671,WASHINGTON,Pacific,24245.0,28.871932,206.228088,Raymond


In [9]:
# Get unique county crime data first
county_crime_unique = county_crime_pop[['state', 'county', 'violent_100k', 'property_100k']].drop_duplicates()

# Join with df_address to get county for each property
address_county = df_address[['zpid', 'city', 'county', 'state']].drop_duplicates()

sql = """
select f.zpid, 
       coalesce(f.violent_100k, c.violent_100k) as violent_100k, 
       coalesce(f.property_100k, c.property_100k) as property_100k
from final_crime f
left join address_county a on f.zpid = a.zpid
left join county_crime_unique c on a.county = c.county and a.state = c.state
"""
final_final_crime = duckdb.sql(sql).df()
final_final_crime

,zpid,violent_100k,property_100k
0,82335574,252.995370,1766.194090
1,247353699,168.484860,1361.997722
2,86332204,271.431803,904.772676
3,81912931,171.883952,974.009063
4,58378866,272.889809,1957.278630
...,...,...,...
773,449986095,60.532688,330.000781
774,66577073,97.139578,377.263013
775,333494016,39.675005,468.731845
776,331228117,106.836380,426.196740


In [10]:
# add final_final_crime as new table to db
con = duckdb.connect(DB_NAME)
con.execute("CREATE OR REPLACE TABLE crime AS SELECT * FROM final_final_crime")
con.sql("select * from crime where zpid = 98839690").df()

,zpid,violent_100k,property_100k
0,98839690,403.957763,2904.405185


In [11]:
con.close()